In [ ]:
import geopandas as gpd
import xLTS

class xLTSAgent(xLTS.Mixin):
    """
    Agent for calculating interesction Level of Traffic Stress using the xLTS library.
    Inherits from xLTS.Mixin.
    """

    def run(self, input_file, output_file):
        """
        Reads the input geospatial file, processes it, and calculates xLTS values.
        Saves the results to the specified output file.

        :param input_file: Path to the input file (e.g., SHP, GPKG, etc.)
        :param output_file: Path to save the output file (e.g., GeoJSON, GPKG, etc.)
        """
        # Read input file into a GeoDataFrame and fill missing values
        base_gdf = gpd.read_file(input_file).fillna(0)

        # Define columns to be converted to int and float types
        int_columns = [
            'ADT', 'Q85', 'NBLane', 'XDistSig', 'VisLim', 'RampTrig',
            'TwoStage', 'PocketTrig', 'FeuxVelo', 'SAS_Velo', 'AvancFlech'
        ]
        float_columns = ['VolRT', 'XileStop', 'c_SENS_CIR']

        # Convert columns to appropriate data types
        base_gdf[int_columns] = base_gdf[int_columns].astype(int)
        base_gdf[float_columns] = base_gdf[float_columns].astype(float)

        # Print the number of crossings
        print(f"Number of crossings: {len(base_gdf)}")

        # Apply xLTS calculation to each row
        base_gdf['xlts'] = base_gdf.apply(
            lambda row: self.xLTS_calc(
                control=row['Control'],
                pocket_trigger=row['PocketTrig'],
                crossing_distance_sig=row['XDistSig'],
                two_stage=row['TwoStage'],
                link_type=row['LinkType'],
                speed_limit=row['Q85'],
                crossing_island=row['XileStop'],
                adt=row['ADT'],
                num_lanes=row['NBLane'],
                visibility_limit=row['VisLim'],
                circular_sensitivity=row['c_SENS_CIR'],
                ramp_trigger=row['RampTrig'],
                bike_signal=row['FeuxVelo'],
                sas_bike=row['SAS_Velo'],
                rt_volume=row['VolRT'],
                avanc_flech=row['AvancFlech']
            ), axis=1
        )

        # Rename specific columns
        base_gdf = base_gdf.rename(columns={
            'f_ONEWAY': 'f_DIVIDED',
            't_ONEWAY': 't_DIVIDED',
            'c_ONEWAY': 'c_DIVIDED'
        })

        # Save the results to the specified output file
        base_gdf.to_file(output_file, driver='GeoJSON')
        # If saving as GPKG, uncomment the following lines and adjust as needed
        # base_gdf.to_file(output_file, layer='xLTS_2', driver='GPKG')

        return base_gdf

###
input_filepath = 'C:\\Users\\natdave\\folders\\Research\\Montreal\\cross_dec16.gpkg'
output_filepath = 'C:\\Users\\natdave\\folders\\Research\\Montreal\\xLTS_2.geojson'

agent = xLTSAgent()
lts_df = agent.run(input_filepath, output_filepath)